In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Ficheros y rutas de entrada/salida 

In [11]:
dir_in = '../../datos/datos_originales/'
file1_in = 'Dataset2.- Precios Semanales Observatorio de Precios Junta de Andalucia.txt'
file2_in = 'Temporada_productos.xlsx'
file3_in = 'Dataset5_Coronavirus_cases.txt'
file4_in = 'consumo_preproc.csv'
dir_out = '../../datos/datos_desarrollo'

## Carga de datos 

In [12]:
df_precios = pd.read_csv(os.path.join(dir_in, file1_in), sep = '|')
df_precios.columns = [columna.lower() for columna in df_precios.columns]
df_temporadas = pd.read_excel(os.path.join(dir_out, file2_in))
df_coronavirus = pd.read_csv(os.path.join(dir_in, file3_in), sep = '|')

**02_precios y temporadas**

In [13]:
df_precio_temporada=df_precios.merge(df_temporadas, how="left", on="producto")

In [14]:
df_consumo = pd.read_csv(os.path.join(dir_out, file4_in), sep = ';')

df_consumo.loc[df_consumo.producto.eq('Champiñoneso setas'), 'producto'] = 'Champiñones o setas'
df_consumo.loc[df_consumo.producto.eq('Omates'), 'producto'] = 'Tomates'
df_consumo.producto = df_consumo.producto.str.split('/', 1).str.get(0).str.lstrip('Otras').str.lstrip().str.capitalize()
df_consumo.loc[df_consumo.producto.eq('Verd '), 'producto'] = 'Verduras'

In [15]:
df_coronavirus['dateRep'] = pd.to_datetime(df_coronavirus['dateRep'])

In [16]:
conditions = [
    df_coronavirus['dateRep'].between("2020-03-01", "2020-06-01"),
    df_coronavirus['dateRep'].between("2020-06-02", "2020-09-01"),
    df_coronavirus['dateRep'].between("2020-09-02", "2020-12-10"),
    (df_coronavirus['dateRep'] > "2020-12-10")
    ]

values = ['Primera ola', 'Periodo de recuperación', 'Segunda ola', 'Tercera ola']
values2 = ['Confinamiento', 'Nueva normalidad', 'Vuelta al trabajo', 'Navidad']
df_coronavirus['Olas'] = np.select(conditions, values)
df_coronavirus['Motivo_ola'] = np.select(conditions, values2)

In [17]:
df_consumo_temp = df_consumo.merge(df_temporadas, on = 'producto', how = 'inner')
df_consumo_temp.to_csv(os.path.join(dir_out, 'consumo_final.csv'), sep = ';', index = False)